In [11]:
!pip install langchain

In [12]:
!pip install langchain-groq

In [13]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature = 0,
    groq_api_key = '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',
    model_name = "llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was....")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [14]:
!pip install chromadb

In [16]:
!pip install langchain-community


   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
    --------------------------------------- 0.0/2.4 MB 325.1 kB/s eta 0:00:08
    --------------------------------------- 0.0/2.4 MB 325.1 kB/s eta 0:00:08
    --------------------------------------- 0.0/2.4 MB 325.1 kB/s eta 0:00:08
    --------------------------------------- 0.0/2.4 MB 325.1 kB/s eta 0:00:08
   - -------------------------------------- 0.1/2.4 MB 327.3 kB/s eta 0:00:07
   - -------------------------------------- 0.1/2.4 MB 327.3 kB/s eta 0:00:07
   - -------------------------------------- 0.1/2.4 MB 327.3 kB/s eta 0:00:07
   - -------------------------------------- 0.1/2.4 MB 227.6 kB/s eta 0:00:10
   - -------------------------------------- 0.1/2.4 MB 226.4 kB/s eta 0:00:10
   -- ------------------------------------- 0.1/2.4 MB 257.2 kB/s eta 0:00:09
   -- -------

In [96]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-38030?from=job%20search%20funnel")

page_data = loader.load().pop().page_content
print(page_data)

Apply for NIKE, Inc. Data & Machine Learning Engineer Undergraduate Internship

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsInternshipsNIKE, Inc. Data & Machine Learning Engineer Und

In [98]:
from langchain_core.prompts import PromptTemplate
import json
prompt_extract = PromptTemplate.from_template(
    """
    ###SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTIONS:
    The scrapped texxt is from the careers page of a website.
    your job is to extract  thr job postings and return them in JSON format containing the 
    following keys; 'role','experience','skills' and 'discription'.
    only retur the valid JSON.
    ###VALID JSON 
    NO PREAMBLE  STICTLY
    NO extra text other than josn file:
    """
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)



```
{
  "role": "NIKE, Inc. Data & Machine Learning Engineer Undergraduate Internship",
  "experience": "0-2 years",
  "skills": [
    "Python",
    "SQL",
    "Tableau",
    "AI/ML techniques",
    "Neural networks",
    "Tree ensembles",
    "Regressions",
    "Data structures",
    "Algorithms",
    "Solution and technical design"
  ],
  "description": "Apply machine learning and engineering expertise to improve existing algorithms and understand and be able to leverage core concepts of AI/ML to research and develop new algorithms. Collaborate in multi-functional teams to support other groups who will also be working on science or science-engineering-related work."
}
```


In [99]:
type(res.content)

str

In [100]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'NIKE, Inc. Data & Machine Learning Engineer Undergraduate Internship',
 'experience': '0-2 years',
 'skills': ['Python',
  'SQL',
  'Tableau',
  'AI/ML techniques',
  'Neural networks',
  'Tree ensembles',
  'Regressions',
  'Data structures',
  'Algorithms',
  'Solution and technical design'],
 'description': 'Apply machine learning and engineering expertise to improve existing algorithms and understand and be able to leverage core concepts of AI/ML to research and develop new algorithms. Collaborate in multi-functional teams to support other groups who will also be working on science or science-engineering-related work.'}

In [101]:
type(json_res)


dict

In [104]:
import pandas as pd
df = pd.read_csv("tech_stack_exam_portfolio.csv")
df


,Tech Stack,Exam Portfolio Links
0,Python,link_to_python_exam_portfolio
1,C#,link_to_csharp_exam_portfolio
2,ASP.NET Core,link_to_aspnetcore_exam_portfolio
3,MERN,link_to_mern_exam_portfolio
4,Machine Learning,link_to_ml_exam_portfolio
5,Big Data,link_to_bigdata_exam_portfolio
6,Data Visualization,link_to_dataviz_exam_portfolio
7,NumPy,link_to_numpy_exam_portfolio
8,Pandas,link_to_pandas_exam_portfolio
9,Scikit-learn,link_to_sklearn_exam_portfolio


In [105]:
import chromadb
import uuid
client = chromadb.PersistentClient('vectrostore')
collection = client.get_or_create_collection(name="portfolio")
if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents = row["Tech Stack"],
                       metadatas = {"links":row["Exam Portfolio Links"]},
                       ids = [str(uuid.uuid4())])

In [106]:
links = collection.query(query_texts=["Experience in python , expertice in React"], n_results = 2).get("metadatas",[])
links

[[{'links': 'link_to_python_exam_portfolio'},
  {'links': 'link_to_tensorflow_exam_portfolio'}]]

In [108]:
job = json_res
job["skills"]

['Python',
 'SQL',
 'Tableau',
 'AI/ML techniques',
 'Neural networks',
 'Tree ensembles',
 'Regressions',
 'Data structures',
 'Algorithms',
 'Solution and technical design']

In [114]:
prompt_email = PromptTemplate.from_template(
    """
    ###JOB DESCRIPTION:
    {job_description}
    ###INSTRUCTION:
    You are jhon , a business developement executive at Crom.ai.Crom.ai is an AI $ Software consultant company 
    the seamless integrtion of business process through automated tools.
    Over out experience,we have empowred numerous enterprises with tailored solutions.
    process optimization,cost reduction,and heightened overall efficiency.
    Your job is to write a cold email to the client regarding the job mentioned above describing the 
    capability in fulfilling their needs.
    Also add the most relevant ones from the following links showcase croms's portfoli :{link_list}
    Remember you are john ,BDE at Crom.ai
    Do not provide preamble.
    ###EMAIL (NO PREAMBLE):
    """
)
chain_email = prompt_email|llm
res = chain_email.invoke({"job_description":str(job),"link_list":links})
print(res.content)

Subject: Expert Data & Machine Learning Engineering Solutions for NIKE, Inc.

Dear Hiring Manager,

I came across the Data & Machine Learning Engineer Undergraduate Internship role at NIKE, Inc. and was impressed by the opportunity to leverage AI/ML expertise to drive innovation. As a Business Development Executive at Crom.ai, I'd like to introduce our company as a trusted partner in delivering tailored AI and software solutions that streamline business processes through automation.

Our team of experts excels in developing and implementing machine learning algorithms, data structures, and solution design. We're confident that our capabilities align with the requirements of the internship role. Our proficiency in Python, SQL, Tableau, and AI/ML techniques, including neural networks, tree ensembles, and regressions, would enable us to make a significant impact at NIKE, Inc.

I'd like to highlight a few examples of our work that demonstrate our expertise in machine learning and data engi